In [33]:
#!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"
#!python -m pip install sentence-transformers

In [1]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import numpy as np

In [24]:
qd_client = QdrantClient("http://localhost:6333")

### Q1. Embedding the query

In [2]:
#EMBEDDING_DIMENSIONALITY = 512
embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [3]:
query = 'I just discovered the course. Can I join now?'
embedding = next(embedding_model.embed([query]))
embedding_array = np.array(embedding)
print("minimal vlaue:", np.min(embedding_array))

minimal vlaue: -0.11726373885183883


#### Cosine similarity

In [10]:
text1 = "I just discovered the course. Can I join now?"
text2 = "I am intersted to learn LLM"

vec1 = next(embedding_model.embed([text1]))
vec2 = next(embedding_model.embed([text2]))

#cosine similarity 
cos_sim = np.dot(vec1, vec2)
print("Cosine similarity:", cos_sim)


Cosine similarity: 0.808900477490305


In [11]:
print("Norm vec1:", np.linalg.norm(vec1))
print("Norm vec2:", np.linalg.norm(vec2)) 

Norm vec1: 1.0
Norm vec2: 0.9999999999999999


In [12]:
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer('jinaai/jina-embeddings-v2-small-en')
# query = 'I just discovered the course. Can I join now?'
# embedding = embedding_model.encode(query)
# embedding_array = np.array(embedding)
# print("minimal vlaue:", np.min(embedding_array))

### Q2. Cosine similarity with another vector

In [13]:
query = "I just discovered the course. Can I join now?"
doc = "Can I still join the course after the start date?"

# Generate embeddings (normalized vectors)
query_vec = next(embedding_model.embed([query]))
doc_vec = next(embedding_model.embed([doc]))

# Cosine similarity = dot product (since vectors are normalized)
cos_sim = np.dot(query_vec, doc_vec)
print("Cosine similarity:", cos_sim)

Cosine similarity: 0.9008528895674548


### Q3. Ranking by cosine

In [16]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [17]:
embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
query = 'I just discovered the course. Can I join now?'
query_vector = next(embedding_model.embed([query]))

# Embed all document texts
doc_texts = [doc["text"] for doc in documents]
doc_vectors = list(embedding_model.embed(doc_texts))

# Convert to numpy array
V = np.vstack(doc_vectors)  
q = np.array(query_vector)  

# Compute cosine similarities
cosine_similarities = V.dot(q)  

# Find the index of the highest similarity
similar_index = int(np.argmax(cosine_similarities))
print("document index with the highest similarity:", similar_index)

document index with the highest similarity: 1


### Q4. Ranking by cosine, version two

In [20]:
# Embed query
embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
query = 'I just discovered the course. Can I join now?'
query_vector = next(embedding_model.embed([query]))

# Embed full_text = question + text
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
doc_vectors = list(embedding_model.embed(full_texts))

# Convert to numpy array
V = np.vstack(doc_vectors)  
q = np.array(query_vector)  

# Compute cosine similarities
cosine_similarities = V.dot(q)  

# Show similarities
for i, score in enumerate(cosine_similarities):
    print(f"Doc {i} cosine similarity: {score:.4f}")

# Best match
best_index = int(np.argmax(cosine_similarities))
print("\nMost similar doc index:", best_index)

Doc 0 cosine similarity: 0.8515
Doc 1 cosine similarity: 0.8437
Doc 2 cosine similarity: 0.8408
Doc 3 cosine similarity: 0.7755
Doc 4 cosine similarity: 0.8086

Most similar doc index: 0


##### question: Is it different from Q3? If yes, why?
IN Q3, embedded only the text field of each document.

and in Q4, embedded question + text (combined as full_text) for each document. This gives the embedding model more context, helping it better understand the document's intent and closely matches the query.

### Q5. Selecting the embedding model

In [22]:
#BAAI/bge-small-en  = 384
embedding_model1 = TextEmbedding(model_name="BAAI/bge-small-en")

text = "Can I still join the course after the start date?"

# Embed the text
embedding1 = next(embedding_model1.embed([text]))
print(" smallest dimensionality for 'BAAI/bge-small-en' model:", np.array(embedding1).shape)

 smallest dimensionality for 'BAAI/bge-small-en' model: (384,)


### Q6. Indexing with qdrant (2 points)

In [23]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [32]:
import uuid
## using BAAI/bge-small-en model 
embedding_BAAI_model = TextEmbedding(model_name="BAAI/bge-small-en")

# Create collection (if not exists)
collection_name = "ml_zoomcamp_faq"
vector_size = 384
qd_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE)
)

# insert documents
payloads = []
vectors = []
ids = []

for doc in documents:
    text = doc["question"] + " " + doc["text"]
    vector = next(embedding_BAAI_model.embed([text]))
    payloads.append(doc)
    vectors.append(vector)
    ids.append(str(uuid.uuid4()))  # unique ID for each doc

qd_client.upsert(
    collection_name=collection_name,
    points=models.Batch(
        ids=ids,
        vectors=vectors,
        payloads=payloads
    )
)

# Query with Q1
query = "I just discovered the course. Can I join now?"
query_vector = next(embedding_BAAI_model.embed([query]))

search_result = qd_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1
)

# highest score
print("highest score:", search_result[0].score)

/tmp/ipykernel_28346/2312395785.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qd_client.recreate_collection(


highest score: 0.8703172


/tmp/ipykernel_28346/2312395785.py:38: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qd_client.search(


using jinaai/jina-embeddings-v2-small-en

In [30]:
## using jinaai/jina-embeddings-v2-small-en
embedding_jina_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Create collection (if not exists)
collection_name = "ml_zoomcamp_faq"
vector_size = 512
qd_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE)
)

# insert documents
payloads = []
vectors = []
ids = []

for doc in documents:
    text = doc["question"] + " " + doc["text"]
    vector = next(embedding_jina_model.embed([text]))
    payloads.append(doc)
    vectors.append(vector)
    ids.append(str(uuid.uuid4()))  # unique ID for each doc

qd_client.upsert(
    collection_name=collection_name,
    points=models.Batch(
        ids=ids,
        vectors=vectors,
        payloads=payloads
    )
)

# Query with Q1
query = "I just discovered the course. Can I join now?"
query_vector = next(embedding_jina_model.embed([query]))

search_result = qd_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1
)

# highest score
print("highest score:", search_result[0].score)

/tmp/ipykernel_28346/3706720415.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qd_client.recreate_collection(


highest score: 0.8620738


/tmp/ipykernel_28346/3706720415.py:36: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qd_client.search(
